In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import StanfordTagger
from src.data.import_data import import_data
from src.data.clean_data import data_cleansing
from nltk.stem import WordNetLemmatizer
from HanTa import HanoverTagger as ht

#### Load data into Dataframe

In [ ]:
# import data
df=import_data()

#### Data Cleansing: Tokenize by white spaces, lowercase, stopword removal

In [ ]:
### get first 10.000 entries to test and get features
df_selected = df.sort_index().loc[0:10000]

In [ ]:
pd.set_option('display.max_colwidth', 50)
df_selected

In [ ]:
df_sel=data_cleansing(df_selected)

In [ ]:
df_sel

#### Feature Generation

In [ ]:
# import string library function 
import string 
#get number of punctuation marks as feature
df_selected['PM_eng']=df_selected['English'].apply(lambda x: len([word for word in x if word in string.punctuation]))
df_selected['PM_ger']=df_selected['German'].apply(lambda x: len([word for word in x if word in string.punctuation]))
#get differences of punctuation marks absolute and relative with respect to smaller number as base
df_selected['PM_dif_abs']=np.abs(df_selected['PM_eng']-df_selected['PM_ger'])
df_selected['PM_dif_rel']=df_selected['PM_dif_abs']/np.minimum(df_selected['PM_eng'],df_selected['PM_ger'])
#set NaNs to zero
df_selected['PM_dif_rel'] = df_selected['PM_dif_rel'].replace(np.nan, 0)
#get number of words as feature
df_selected['Words_eng']=df_selected['English'].apply(lambda x: len([word for word in x if word not in string.punctuation]))
df_selected['Words_ger']=df_selected['German'].apply(lambda x: len([word for word in x if word not in string.punctuation]))
#get differences of words absolute and relative with respect to smaller number as base
df_selected['Words_dif_abs']=np.abs(df_selected['Words_eng']-df_selected['Words_ger'])
df_selected['Words_dif_rel']=df_selected['Words_dif_abs']/np.minimum(df_selected['Words_eng'],df_selected['Words_ger'])
#set NaNs to zero
df_selected['Words_dif_rel'] = df_selected['Words_dif_rel'].replace(np.nan, 0)

In [ ]:
#number of question marks
marks='?'
df_selected['ger_ques']=df_selected['German'].apply(lambda x: len([word for word in x if word in marks]))
df_selected['eng_ques']=df_selected['English'].apply(lambda x: len([word for word in x if word in marks]))
#number of exclamation marks
marks='!'
df_selected['ger_exc']=df_selected['German'].apply(lambda x: len([word for word in x if word in marks]))
df_selected['eng_exc']=df_selected['English'].apply(lambda x: len([word for word in x if word in marks]))


In [ ]:
#get number of characters in words and the average char per word
df_selected['char_eng']=df_selected['English'].apply(lambda x: len(str([word for word in x if word not in string.punctuation])))
df_selected['char_eng_avg']=df_selected['char_eng']/df_selected['Words_eng']
df_selected['char_ger']=df_selected['German'].apply(lambda x: len(str([word for word in x if word not in string.punctuation])))
df_selected['char_ger_avg']=df_selected['char_ger']/df_selected['Words_ger']
#absolute difference between avg
df_selected['char_avg_abs']=df_selected['char_ger_avg']-df_selected['char_eng_avg']

In [ ]:
#use pos-tagger and get number of nouns, verbs, adjectives, tagset universal to only get the highest categories and 
df_selected['English_pos']=df_selected.apply(lambda x: nltk.pos_tag(x['English'],tagset='universal'), axis=1)

#pos-tagging with nltk not suppored in german yet -> use pretrained model for german as tagger
##pretrained model for german as tagger
#Christian Wartena (2019). A Probabilistic Morphology Model for German Lemmatization. In: Proceedings of the 15th Conference on Natural Language Processing (KONVENS 2019): Long Papers. Pp. 40-49, Erlangen.

tagger = ht.HanoverTagger('morphmodel_ger.pgz')
ger_pos=[]
for i in df_selected.itertuples():
    ger_pos.append(list(map(lambda x: tagger.analyze(x),i.German)))
df_selected['German_pos']=ger_pos

In [ ]:
# df_selected.to_csv('Feature_Generation.csv',index=False)

In [ ]:
df_selected=pd.read_csv('Feature_Generation.csv')

In [ ]:
df_selected

In [ ]:
universal_pos=['ADJ','ADP','ADV','CONJ','DET','NOUN','NUM','PRT','PRON','VERB','X']
for u_tag in universal_pos:
    df_selected[f'en_{u_tag}'] = df_selected['English_pos'].apply(lambda row: nltk.FreqDist(tag for (word, tag) in row if tag==u_tag)[u_tag])